<span style="font-size:32px; font-family: 'Arial">**Micrograd - Backpropagation Engine**</span>

<span style="font-size:22px; font-family: 'Arial'">Micrograd is a lightweight Python library that focuses on automatic differentiation, a key technique for training neural networks through backpropagation. It helps compute gradients automatically, making it easier to optimize neural networks by adjusting weights and biases based on the error (loss) calculated during training. Micrograd is minimalist and designed for efficiency, offering foundational capabilities for implementing custom neural network architectures</span>


In [1]:
import math
import random

In [2]:
class Value:
    
    def __init__(self, data, inputs =(), oper="",label=""):
        self.label = label
        self.data = data
        self.grad = 0.0
        # backward - updates the gradients of the inputs( objects used to create) of the current object
        self.backward = lambda:None 
        # inputs - contains the inputs(or objects) which are use to create the current object
        self.inputs = set(inputs)
        # oper - mathematical operation used to create the current object
        self.oper = oper 
        """ inputs(or objects) which are used to create the new
            object are being stored in order to enable
            'Backwardpass' """
    
    # string representation of a class object/instance
    def __repr__(self): 
        return f"(Value={self.data})"
    
    # self+other = self.__add__(other)
    def __add__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        new_object = Value(data=(self.data + other.data), 
                           inputs= (self, other), 
                           oper="+")
        def backward():
            self.grad += (1.0)*(new_object.grad)
            other.grad += (1.0)*(new_object.grad)
            
        new_object.backward = backward
        return new_object
    
    # invoked when __add__() doesn't works that is object is second operand (eg- 2+a)
    def __radd__(self,other):
        return self+other
    
    # self-other = self.__sub__(other)
    def __sub__(self,other):
        other = other if isinstance(other,Value) else Value(other)
        new_object = Value(data=(self.data - other.data),
                           inputs= (self, other),
                           oper="-")
        
        def backward():
            self.grad += (1.0)*(new_object.grad)
            other.grad += (-1.0)*(new_object.grad)
        
        new_object.backward = backward
        return new_object
    
    # invoked when __sub__() doesn't works that is object is second operand (eg- 2-a)
    def __rsub__(self,other): 
        return (-self)+other
    
    # self*other = self.__mul__(other)
    def __mul__(self,other): 
        other = other if isinstance(other,Value) else Value(other)
        new_object = Value(data=(self.data * other.data), 
                           inputs= (self, other), 
                           oper="*") 
        def backward():
            self.grad += (other.data)*(new_object.grad)
            other.grad += (self.data)*(new_object.grad)
        
        new_object.backward = backward
        return new_object

    # invoked when __add__() doesn't works that is object is second operand (eg- 2*a)
    def __rmul__(self,other): 
        return self*other
    
    # self/div = self.__div__(other)
    def __truediv__(self,other): 
        return self*(other**-1)
    
    # -self = self.__neg__()
    def __neg__(self):
        return self*(-1)
    
    # self**other = self.__pow__(other)
    def __pow__(self,other): 
        assert isinstance(other, (int, float)), "Error: only supports int/float for power"
        new_object = Value(data=(self.data**other),
                           inputs=(self,),
                           oper=f"**{other}")
        
        # dL/dx = (dx^p/dx) * (dL/dx^p)
        def backward():
           self.grad += (other)*(self.data**(other-1))*(new_object.grad) 
        new_object.backward = backward 
        return new_object
        
    
    def tanh(self):
        tan_h = (math.exp(2*self.data)-1)/(math.exp(2*self.data)+1)
        new_object = Value(data=(tan_h),
                           inputs=(self,),
                           oper="tanh")
        def backward():
           self.grad += (1-tan_h**2)*(new_object.grad)
        new_object.backward = backward 
        return new_object
    
    def exp(self):
        new_object = Value(data=(math.exp(self.data)),
                           inputs=(self,),
                           oper="exp")
        def backward():
            self.grad += (new_object.data)*(new_object.grad)
        new_object.backward = backward
        return new_object

    def log(self):
        new_object = Value(data=(math.log(self.data)),
                           inputs=(self,),
                           oper="log")
        def backward():
            self.grad += (1.0/self.data)*(new_object.grad)
        new_object.backward = backward
        return new_object
    
    def backward_pass(self):
        topo = []
        visited = set()
        # topological sort as our neural network is DAG
        def build_topo(node):
            if node not in visited:
                visited.add(node)
                for input in node.inputs:
                    build_topo(input) # DFS
                topo.append(node)
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node.backward()